In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from bigdagafriend import *
try:
    sc.stop()
except:
    pass
spark = SparkSession \
    .builder.enableHiveSupport() \
    .appName("Python Spark SQL") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
df = spark.read.json("/Users/tq/Downloads/logics_shard1009.07.03.2017.log")

In [3]:
df.createOrReplaceTempView('RawTable')
preparetable = 'RawTable'
result1 = spark.sql(u'''
SELECT
    *
FROM %s
WHERE accountid is not null 
AND type_name!="CorpLevelChg" 
'''%preparetable).cache().createOrReplaceTempView('jsonTable')

In [4]:
result2 = spark.sql(u"""
SELECT
    jsonTable.userid as uid,
    jsonTable.accountid AS acid,
    jsonTable.gid AS gid,
    jsonTable.sid AS sid,
    jsonTable.channel AS ch,
    jsonTable.corplvl AS corpLvl,
    jsonTable.info.VipLvl AS vipLvl,
    jsonTable.info.Money AS payNum,
    jsonTable.info.PayTime AS payTime,
    SUBSTRING(jsonTable.logtime,0,10) AS LogTime,
    CAST(jsonTable.utc8 as DATE) AS date
FROM jsonTable
WHERE type_name = "IAP"
""").cache().createOrReplaceTempView('account_payTime')

In [5]:
result = spark.sql(u"""
SELECT * 
FROM account_payTime
""")

In [6]:
result.show()

+--------------------+--------------------+---+----+------------+-------+------+------+--------------+----------+----------+
|                 uid|                acid|gid| sid|          ch|corpLvl|vipLvl|payNum|       payTime|   LogTime|      date|
+--------------------+--------------------+---+----+------------+-------+------+------+--------------+----------+----------+
|f6d38e9a-b775-4e6...|200:1009:f6d38e9a...|200|1009|130134005400|     88|    15|  9800|20170307092157|1488849770|2017-03-07|
|f6d38e9a-b775-4e6...|200:1009:f6d38e9a...|200|1009|130134005400|     88|    15|   100|20170307092237|1488849770|2017-03-07|
|f6d38e9a-b775-4e6...|200:1009:f6d38e9a...|200|1009|130134005400|     88|    15|   100|20170307092252|1488849783|2017-03-07|
|d7566633-063f-4bd...|200:1009:d7566633...|200|1009|130134005400|     25|     0|  3000|20170307130531|1488863157|2017-03-07|
|2b5d190e-a67f-4c2...|200:1009:2b5d190e...|200|1009|130134001500|     66|    10|  9800|20170307135722|1488866391|2017-03-07|


In [15]:
x = result.replace(["130134005400","130134000001"], ["0","1"], "ch")

In [16]:
print type(x)

<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
x.toPandas()

,uid,acid,gid,sid,ch,corpLvl,vipLvl,payNum,payTime,LogTime,date
0,f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200:1009:f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200,1009,0,88,15,9800,20170307092157,1488849770,2017-03-07
1,f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200:1009:f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200,1009,0,88,15,100,20170307092237,1488849770,2017-03-07
2,f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200:1009:f6d38e9a-b775-4e65-8a9f-5b545ae41e67,200,1009,0,88,15,100,20170307092252,1488849783,2017-03-07
3,d7566633-063f-4bde-bb49-9c29c6a5e168,200:1009:d7566633-063f-4bde-bb49-9c29c6a5e168,200,1009,0,25,0,3000,20170307130531,1488863157,2017-03-07
4,2b5d190e-a67f-4c2c-a588-17ba6c468663,200:1009:2b5d190e-a67f-4c2c-a588-17ba6c468663,200,1009,130134001500,66,10,9800,20170307135722,1488866391,2017-03-07
5,2b5d190e-a67f-4c2c-a588-17ba6c468663,200:1009:2b5d190e-a67f-4c2c-a588-17ba6c468663,200,1009,130134001500,66,10,100,20170307140248,1488866641,2017-03-07
6,ee98bba8-9e9b-4db2-93aa-823127b924c1,200:1009:ee98bba8-9e9b-4db2-93aa-823127b924c1,200,1009,1,85,13,3000,20170307171236,1488877977,2017-03-07
7,ee98bba8-9e9b-4db2-93aa-823127b924c1,200:1009:ee98bba8-9e9b-4db2-93aa-823127b924c1,200,1009,1,85,13,19800,20170308072124,1488928907,2017-03-08
8,ee98bba8-9e9b-4db2-93aa-823127b924c1,200:1009:ee98bba8-9e9b-4db2-93aa-823127b924c1,200,1009,1,85,13,3000,20170308072230,1488928968,2017-03-08
9,3a0685ed-7a9d-4432-8a52-83681b30fea5,200:1009:3a0685ed-7a9d-4432-8a52-83681b30fea5,200,1009,5000,89,16,2500,20170308072507,1488929134,2017-03-08


In [18]:
properties = {
    "user": "test1",
    "password": "QmPhaQ8hYsxx"
}
payTableName = "account_payTime"
url = "jdbc:mysql://54.223.192.252:3306/test"

In [19]:
x.write.jdbc(url, payTableName, "append", properties)